In [6]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patch
import io
from pyspark.sql import SparkSession


# Because we are in local environment some changes are needed
# 1. create Spark session local

spark = SparkSession.builder.master('local').getOrCreate()

metadata_data_dir = "metadata.csv"
volumetric_data_dir = "images/*.pck"
connection_string = "DefaultEndpointsProtocol=http;AccountName=devstoreaccount1;AccountKey=Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw==;BlobEndpoint=http://fhirblob:10000/devstoreaccount1"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
def toImage(mri, metadata):
    filename = mri[0].split("/").pop()
    exam = next((x for x in metadata if x['volumeFilename'] == filename), None)
    volumetric_data = pickle.loads(mri[1])
    # Get all roi slices from volume
    z_start = exam['roiZ']
    depth = exam['roiDepth']
    plot = None
    for z in range(z_start, z_start + depth):
        slice = volumetric_data[z, :, :]
        # Get roi dimensions
        x, y, w, h = [exam[attr] for attr in ['roiX', 'roiY', 'roiWidth', 'roiHeight']]
        # Extract ROI
        roi = slice[y:y+h, x:x+w]
        # Plot slice and roi
        plt.ioff()
        figure = plt.figure()
        plot = plt.subplot2grid((1, 4), (0, 0), 1, 3) # This makes the slice plot larger than roi plot
        plot.add_patch(patch.Rectangle((x, y), w, h, fill=None, color='red'))
        plot.imshow(slice, cmap='gray')
        plot = plt.subplot2grid((1, 4), (0, 3), 1, 1)
        plot.imshow(roi, cmap='gray')
    buf = io.BytesIO()
    plot.figure.savefig(buf, format="png")
    plot.clf()
    buf.seek(0)
    blob_client = blob_service_client.get_blob_client(container='images', blob=filename.replace("pck", "png"))
    blob_client.upload_blob(buf, overwrite=True)
    buf.close()
    
metadata = spark.read.csv(metadata_data_dir, header='true', inferSchema='true').collect()
mris = spark.sparkContext.binaryFiles(volumetric_data_dir).collect()
for mri in mris:
    toImage(mri, metadata)
print("FinishJob!")

<ipython-input-6-de5cff312aed>:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure = plt.figure()


FinishJob!


In [10]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, PublicAccess,AccessPolicy, ContainerSasPermissions
from datetime import datetime, timedelta
connection_string = "DefaultEndpointsProtocol=http;AccountName=devstoreaccount1;AccountKey=Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw==;BlobEndpoint=http://fhirblob:10000/devstoreaccount1"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client("images")
# Create access policy
access_policy = AccessPolicy(permission=ContainerSasPermissions(read=True, write=True),
                             expiry=datetime.utcnow() + timedelta(hours=1),
                             start=datetime.utcnow() - timedelta(minutes=1))
identifiers = {'read': access_policy}
# Specifies full public read access for container and blob data.
public_access = PublicAccess.Container
# Set the access policy on the container
container_client.set_container_access_policy(signed_identifiers=identifiers, public_access=public_access)

{'etag': '"0x1C45B376751DB40"',
 'last_modified': datetime.datetime(2020, 10, 13, 10, 30, 27, tzinfo=datetime.timezone.utc),
 'client_request_id': '1c1c82ac-0d3f-11eb-9d34-0242ac120005',
 'request_id': 'd4bfb116-f57a-4c34-b185-025a5972d259',
 'version': '2020-02-10',
 'date': datetime.datetime(2020, 10, 13, 10, 30, 27, tzinfo=datetime.timezone.utc),
 'error_code': None}